In [10]:
from math import exp

In [36]:
def make_group(NV,NH=1,b=75,D=100):
    if NV < 2:
        raise ValueError("Number of bolts per vertical row must be at least 2.")
    if NH < 1:
        raise ValueError("Number of vertical rows must be at least 1.")
    d = D/(NH-1.) if NH > 1 else 0.
    ylo = -(NV-1)*b/2.
    xlo = -(NH-1)*d/2.
    ans = []
    for j in range(0,NH):
        for i in range(0,NV):
            x = xlo
            y = ylo + i*b
            ans.append(((i,j),x,y))
        xlo += d
    return ans

In [37]:
make_group(4,2)

[((0, 0), -50.0, -112.5),
 ((1, 0), -50.0, -37.5),
 ((2, 0), -50.0, 37.5),
 ((3, 0), -50.0, 112.5),
 ((0, 1), 50.0, -112.5),
 ((1, 1), 50.0, -37.5),
 ((2, 1), 50.0, 37.5),
 ((3, 1), 50.0, 112.5)]

In [42]:
def compute1(grp,L=300,r0=None):
    if r0 is None:
        r0 = L/3.
    rmax = 0.
    bolts = []
    for id,x,y in grp:
        x = x + r0
        r = (x*x + y*y)**0.5
        rmax = max(rmax,r)
        bolts.append((id,x,y,r))

#    Ru = 282.
    Ru = 329.
    mu = 0.4   # for delta in mm, 10 for delta in in.
    lamda = 0.55
    delta_max = 8.64
    
    sumM = 0.
    sumV = 0.
    for id,x,y,r in bolts:
        delta = (r/rmax)*delta_max
        R = Ru*(1 - exp(-mu*delta))**lamda
        sumM += R*r
        sumV += R*(x/r)
    P = sumM/(L+r0)
    return sumM,sumV,P,r0

In [43]:
grp = make_group(6)
display(grp)
display(compute1(grp,300,80))

[((0, 0), 0.0, -187.5),
 ((1, 0), 0.0, -112.5),
 ((2, 0), 0.0, -37.5),
 ((3, 0), 0.0, 37.5),
 ((4, 0), 0.0, 112.5),
 ((5, 0), 0.0, 187.5)]

(268285.413194336, 1132.7524455019975, 706.0142452482527, 80)

In [44]:
compute1(grp,300,36.4)

(222344.834244154, 661.980969970487, 660.9537284308977, 36.4)

In [45]:
compute1(grp,300,0)

(206805.3595790053, 0.0, 689.3511985966844, 0)

In [46]:
def compute(grp,L=300):
    M,V,P,r0 = compute1(grp,L,L/2.)
    print(M,V,P,r0)
    F0 = V-P
    M,V,P,r1 = compute1(grp,L,L/3.)
    print(M,V,P,r1)
    F1 = V-P
    r = [r0,r1]
    F = [F0,F1]
    while True:
        F0,F1 = F[-2:]
        r0,r1 = r[-2:]
        #print(r0,F0,r1,F1)
        if len(r) > 100:
            break
        if abs(F1) < 1.:
            break
        r2 = r1 - F1*(r1-r0)/(F1-F0)
        M,V,P,r2 = compute1(grp,L,r2)
        F2 = V-P
        F.append(F2)
        r.append(r2)
        print(r2,V,P,F2)

In [47]:
compute(grp,300)

369510.2316626441 1509.9291615488683 821.133848139209 150.0
294791.35239908285 1274.0307687360041 736.9783809977072 100.0
-76.96126042192566 -1108.0279799516695 1185.8589608237235 -2293.8869407753928
66.42899885223937 1014.4662146050545 687.4871109367172 326.9791036683373
48.539637985576576 822.3117079974338 668.4395234117299 153.87218458570396
32.638049233474725 606.3893694523224 659.8703355370525 -53.480966084730085
36.739420859585955 666.8508364340338 661.0836325552754 5.767203878758437
36.34019424440248 661.1204441510188 660.9313712452766 0.18907290574213675


grp2 = make_group(3,2,80,80)
grp2

In [48]:
compute(grp2,200)

226989.71462290015 1525.212124496256 756.6323820763338 100.0
178011.06788067956 1253.97748477483 667.5415045525483 66.66666666666667
-40.654417358150994 -815.2048023683975 933.6499986872898 -1748.8548010556874
39.71629902372584 742.8848080960062 617.6025004012629 125.28230769474328
34.34367779319341 585.4683620125747 618.2316919002225 -32.76332988764773
35.45743812489222 611.2271814287443 617.7561041545059 -6.528922725761618
35.73461820736211 617.8831529271571 617.6623150008397 0.22083792631735832
